In [ ]:
library(eivtools)

In [ ]:
set.seed(1001)
## simulate data with covariates x1, x2 and z.
.n <- 1000
.d <- data.frame(x1 = rnorm(.n))
.d$x2 <- sqrt(0.5)*.d$x1 + rnorm(.n, sd=sqrt(0.5))
.d$z <- as.numeric(.d$x1 + .d$x2 > 0)
## generate outcome
## true regression parameters are c(2,1,1,-1)
.d$y <- 2.0 + 1.0*.d$x1 + 1.0*.d$x2 - 1.0*.d$z + rnorm(.n)
## generate error-prone covariates w1 and w2
Sigma_error <- diag(c(0.20, 0.30))
dimnames(Sigma_error) <- list(c("w1","w2"), c("w1","w2"))
.d$w1 <- .d$x1 + rnorm(.n, sd = sqrt(Sigma_error["w1","w1"]))
.d$w2 <- .d$x2 + rnorm(.n, sd = sqrt(Sigma_error["w2","w2"]))

In [ ]:
Sigma_error

In [ ]:
set.seed(3001)
cat("NOTE: this example uses MCMC and takes a little while to run\n")
## example of estimating school "value-added" effects on math test scores,
## adjusting for lag 1 math and ELA scores and accounting for the
## heteroskedastic measurement errors in those scores.
data(testscores)
print(length(unique(testscores$schoolid)))
## to help interpretation of model coefficients and school effects, standardize
## current and lag 1 test scores to have mean zero and variance 1. Also adjust
## the conditional standard errors of measurement for the lag 1 scores.
testscores$math <- as.vector(scale(testscores$math))
testscores$math_lag1_csem <- testscores$math_lag1_csem / sd(testscores$math_lag1)
testscores$math_lag1 <- as.vector(scale(testscores$math_lag1))

testscores$lang_lag1_csem <- testscores$lang_lag1_csem / sd(testscores$lang_lag1)
testscores$lang_lag1 <- as.vector(scale(testscores$lang_lag1))
## create pieces needed to call lr_ancova. Note that first column of Z
## must be an intercept.
outcome_model <- "normal"
Y <- testscores$math
W <- testscores[,c("math_lag1","lang_lag1")]
Z <- cbind(1, testscores[,c("sped","frl")])
G <- testscores$schoolid
## create varfuncs. Need to be careful to pass conditional measurement error
## variances, which require squaring the CSEMs
varfuncs <- list()
tmp <- unique(testscores[,c("math_lag1","math_lag1_csem")])
names(tmp) <- c("x","gx")
tmp <- tmp[order(tmp$x),]
tmp$gx <- tmp$gx^2
varfuncs[[1]] <- list(type="log_polynomial", vtab=tmp)
tmp <- unique(testscores[,c("lang_lag1","lang_lag1_csem")])
names(tmp) <- c("x","gx")
tmp <- tmp[order(tmp$x),]
tmp$gx <- tmp$gx^2
varfuncs[[2]] <- list(type="log_polynomial", vtab=tmp)

In [ ]:
varfuncs